# 📘 Phase 3: Retrieval-Augmented Generation (RAG) with Gemini 2.0


In this phase, we combine semantic retrieval using FAISS with **Gemini 2.0** to generate grounded, high-quality responses to questions on drug targets for depression, psychosis, and anxiety.

We simulate a RAG pipeline that:
1. Accepts a user query.
2. Retrieves top-k relevant text chunks from the FAISS index.
3. Constructs a prompt for Gemini 2.0 including retrieved context.
4. Generates an answer grounded in the retrieved PubMed knowledge.

> **Note**: We simulate Gemini 2.0 here using the `google.generativeai` package.
    

In [ ]:

!pip install -q google-generativeai faiss-cpu sentence-transformers
import google.generativeai as genai
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import os

# Load model and index again (if needed in runtime)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Gemini configuration
genai.configure(api_key="your_google_api_key")  # Replace with your Gemini API Key
gemini = genai.GenerativeModel("gemini-2.0-flash")
    

### 🔄 Load Existing Vector Store

In [ ]:

# Load your existing index (in real-world, index should be stored/persisted)
# Recreate index here for simplicity
text_chunks = [...]  # Replace this with actual list of text chunks from Phase 2
embeddings = model.encode(text_chunks).astype("float32")

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
    

### 🔍 Define Retrieval + Gemini 2.0 RAG Pipeline

In [ ]:

def retrieve_relevant_chunks(query, top_k=5):
    query_vector = model.encode([query]).astype("float32")
    distances, indices = index.search(query_vector, top_k)
    return [text_chunks[i] for i in indices[0]]

def generate_rag_response(query):
    context_chunks = retrieve_relevant_chunks(query)
    context = "\n".join(context_chunks)
    prompt = f"You are a biomedical research assistant helping identify drug targets.\n\nContext:\n{context}\n\nUser Question: {query}\n\nAnswer:"
    
    response = gemini.generate_content(prompt)
    return response.text
    

### 🧪 Test: Drug Targets for Depression

In [ ]:

query = "What are the latest drug targets for treating depression?"
response = generate_rag_response(query)
print("🤖 Gemini 2.0 Response:\n")
print(response)
    